# 1) Titolo e obiettivi
Lezione 37: Introduzione concettuale al Deep Learning con un MLP minimale.

---

## Mappa concettuale della lezione

```
DEEP LEARNING - ARCHITETTURA MLP
=================================

INPUT LAYER         HIDDEN LAYER        OUTPUT LAYER
(features)          (neuroni)           (prediction)

  x₁ ──────┐                      
           ├───►  h₁ = ReLU(W₁x + b₁)
  x₂ ──────┤                            ──────► ŷ = σ(W₂h + b₂)
           ├───►  h₂ = ReLU(W₁x + b₁)
  x₃ ──────┘                      
           └───►  h₃ = ReLU(W₁x + b₁)


FORWARD PASS              BACKWARD PASS (BACKPROP)
==============            ========================

   X ────► W₁,b₁ ────► ReLU ────► W₂,b₂ ────► σ ────► Loss
   
           ◄──── dW₂ ◄──── dσ ◄──── dLoss
           ◄──── dW₁ ◄──── dReLU ◄────────────────────────


TRAINING LOOP
=============
   ┌─────────────────────────────────────────┐
   │  for epoch in range(n_epochs):          │
   │      ŷ = forward(X)         # predict   │
   │      L = loss(y, ŷ)         # evaluate  │
   │      grads = backward(L)    # gradient  │
   │      W -= lr * grads        # update    │
   └─────────────────────────────────────────┘
```

---

## Obiettivi didattici

| # | Obiettivo | Livello |
|---|-----------|---------|
| 1 | Comprendere architettura feed-forward (input → hidden → output) | Fondamentale |
| 2 | Implementare forward pass con attivazioni non lineari | Operativo |
| 3 | Capire backpropagation come chain rule applicata | Concettuale |
| 4 | Confrontare MLP con baseline lineare | Valutativo |
| 5 | Riconoscere overfitting e ruolo della complessita' | Critico |
| 6 | Preparare transizione verso framework (PyTorch, TensorFlow) | Prospettiva |

---

## Concetti chiave

> **Multilayer Perceptron (MLP)**: rete neurale feed-forward con uno o piu' hidden layer; ogni layer applica trasformazione lineare + attivazione non lineare.

> **Backpropagation**: algoritmo che calcola i gradienti della loss rispetto ai pesi applicando la chain rule dal output verso l'input.

> **Attivazione non lineare**: senza ReLU/tanh/sigmoid, composizione di layer lineari rimane lineare - nessun vantaggio su regressione logistica.

---

## Perche' "deep" funziona?

```
MODELLO LINEARE                    MLP (2 layer)
===============                    ==============

    y = Wx + b                     y = W₂ · ReLU(W₁x + b₁) + b₂

    ┌──────────────┐               ┌──────────────┐
    │ Decision     │               │ Decision     │
    │ boundary:    │               │ boundary:    │
    │   LINEA      │               │ CURVA/REGIONE│
    └──────────────┘               └──────────────┘

Non puo' separare                  Puo' separare
dati non linearmente               dati non lineari
separabili (es. XOR)               (es. moons, circles)
```

---

## Cosa useremo
- `NumPy` per implementazione MLP da zero
- `make_moons` per dataset non lineare
- `LogisticRegression` come baseline
- Funzioni di attivazione: ReLU, sigmoid

## Prerequisiti
- Regressione logistica (Lezione 06)
- Derivate e chain rule
- Concetto di loss function (log-loss)


# 2) Teoria concettuale
- Reti feed-forward: composizione di layer lineari + attivazioni non lineari.
- Backpropagation: calcolo gradiente per aggiornare i pesi con discesa del gradiente.
- Bias-variance: reti piu' grandi possono overfittare; serve regolarizzare/early stopping.


# 3) Schema mentale / mappa decisionale
1. Definisci architettura (input-hidden-output) e attivazioni.
2. Inizializza pesi piccoli.
3. Forward -> loss -> backward -> update.
4. Monitora loss train e (se disponibile) valida su hold-out.
5. Confronta con baseline lineare per capire il contributo della non linearita'.


# 4) Sezione dimostrativa
Demo: MLP NumPy su two moons con poche epoche per mostrare convergenza parziale e confronto con modello lineare.


In [ ]:
# Setup
import numpy as np
from sklearn.datasets import make_moons
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
np.random.seed(42)


In [ ]:
# Dataset two moons
X, y = make_moons(n_samples=500, noise=0.2, random_state=42)
X = X.T  # shape (2, n)
y = y.reshape(1, -1)
print(f"Shape X: {X.shape}, y: {y.shape}")
assert X.shape[0] == 2


In [ ]:
# Funzioni di attivazione e helper

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def relu(z):
    return np.maximum(0, z)

def relu_deriv(z):
    return (z > 0).astype(float)


In [ ]:
# Inizializzazione pesi MLP (2 -> 3 -> 1)
nh = 3
W1 = 0.1 * np.random.randn(nh, 2)
b1 = np.zeros((nh, 1))
W2 = 0.1 * np.random.randn(1, nh)
b2 = np.zeros((1, 1))
alpha = 0.1
epochs = 3000


In [ ]:
# Training loop
losses = []
for epoch in range(epochs):
    Z1 = W1 @ X + b1
    A1 = relu(Z1)
    Z2 = W2 @ A1 + b2
    A2 = sigmoid(Z2)

    m = y.shape[1]
    loss = -np.mean(y * np.log(A2 + 1e-8) + (1 - y) * np.log(1 - A2 + 1e-8))
    losses.append(loss)

    dZ2 = A2 - y
    dW2 = (1/m) * dZ2 @ A1.T
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = W2.T @ dZ2
    dZ1 = dA1 * relu_deriv(Z1)
    dW1 = (1/m) * dZ1 @ X.T
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

    W2 -= alpha * dW2
    b2 -= alpha * db2
    W1 -= alpha * dW1
    b1 -= alpha * db1

print(f"Loss finale: {losses[-1]:.3f}")


In [ ]:
# Valutazione MLP vs baseline logistica
A1 = relu(W1 @ X + b1)
A2 = sigmoid(W2 @ A1 + b2)
preds = (A2 > 0.5).astype(int)
acc_mlp = accuracy_score(y.flatten(), preds.flatten())
print(f"Accuracy MLP: {acc_mlp:.3f}")

log_reg = LogisticRegression()
log_reg.fit(X.T, y.flatten())
acc_lin = accuracy_score(y.flatten(), log_reg.predict(X.T))
print(f"Accuracy logistica (lineare): {acc_lin:.3f}")


### Osservazioni
- Il MLP con ReLU approssima il confine non lineare meglio della logistica lineare.
- Poche epoche e nessuna regolarizzazione: possibile overfitting se si aumenta nh/epoche.


# 5) Esercizi svolti (step-by-step)
## Esercizio 37.1 - Variare hidden size
Obiettivo: cambiare `nh` e confrontare la loss finale.


In [ ]:
# Esercizio 37.1
for nh in [2, 4, 8]:
    W1 = 0.1 * np.random.randn(nh, 2)
    b1 = np.zeros((nh, 1))
    W2 = 0.1 * np.random.randn(1, nh)
    b2 = np.zeros((1, 1))
    for _ in range(1000):
        Z1 = W1 @ X + b1
        A1 = relu(Z1)
        Z2 = W2 @ A1 + b2
        A2 = sigmoid(Z2)
        m = y.shape[1]
        dZ2 = A2 - y
        dW2 = (1/m) * dZ2 @ A1.T
        db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
        dA1 = W2.T @ dZ2
        dZ1 = dA1 * relu_deriv(Z1)
        dW1 = (1/m) * dZ1 @ X.T
        db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
        W2 -= alpha * dW2
        b2 -= alpha * db2
        W1 -= alpha * dW1
    loss = -np.mean(y * np.log(A2 + 1e-8) + (1 - y) * np.log(1 - A2 + 1e-8))
    print(f"nh={nh}, loss={loss:.3f}")


## Esercizio 37.2 - Attivazione tanh


In [ ]:
# Esercizio 37.2

def tanh(z):
    return np.tanh(z)

def tanh_deriv(z):
    return 1 - np.tanh(z)**2

W1 = 0.1 * np.random.randn(nh, 2)
b1 = np.zeros((nh, 1))
W2 = 0.1 * np.random.randn(1, nh)
b2 = np.zeros((1, 1))
for _ in range(1000):
    Z1 = W1 @ X + b1
    A1 = tanh(Z1)
    Z2 = W2 @ A1 + b2
    A2 = sigmoid(Z2)
    m = y.shape[1]
    dZ2 = A2 - y
    dW2 = (1/m) * dZ2 @ A1.T
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dA1 = W2.T @ dZ2
    dZ1 = dA1 * tanh_deriv(Z1)
    dW1 = (1/m) * dZ1 @ X.T
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    W2 -= alpha * dW2
    b2 -= alpha * db2
    W1 -= alpha * dW1
loss_tanh = -np.mean(y * np.log(A2 + 1e-8) + (1 - y) * np.log(1 - A2 + 1e-8))
print(f"Loss finale con tanh: {loss_tanh:.3f}")


## Esercizio 37.3 - Baseline lineare


In [ ]:
# Esercizio 37.3
lin = LogisticRegression()
lin.fit(X.T, y.flatten())
acc_lin = accuracy_score(y.flatten(), lin.predict(X.T))
print(f"Accuracy baseline lineare: {acc_lin:.3f}")


# 6) Conclusione operativa - Bignami Deep Learning

---

## I 5 Take-Home Messages

| # | Concetto | Perche' conta |
|---|----------|---------------|
| 1 | **MLP = Linear + Activation stacked** | Attivazioni non lineari danno potere espressivo |
| 2 | **Backprop = chain rule sistematica** | Calcola gradienti per qualsiasi architettura |
| 3 | **Piu' neuroni = piu' capacita' = piu' rischio overfitting** | Bilanciare complessita' con validazione |
| 4 | **Learning rate critico** | Troppo alto diverge, troppo basso non converge |
| 5 | **Baseline lineare obbligatoria** | MLP deve battere logistic/linear per giustificare complessita' |

---

## Architettura MLP - Componenti

```
COMPONENTE          RUOLO                       FORMULA
==========          =====                       =======

Pesi W              Trasformazione lineare      z = Wx + b
Bias b              Offset per ogni neurone     

Attivazione         Non linearita'              ReLU(z) = max(0, z)
                                                σ(z) = 1/(1+e^-z)
                                                tanh(z) = (e^z - e^-z)/(e^z + e^-z)

Loss                Misura errore               L = -[y·log(ŷ) + (1-y)·log(1-ŷ)]

Gradient            Direzione discesa           ∂L/∂W = ∂L/∂ŷ · ∂ŷ/∂z · ∂z/∂W

Update              Aggiornamento pesi          W ← W - η · ∂L/∂W
```

---

## Confronto attivazioni

| Attivazione | Formula | Pro | Contro |
|-------------|---------|-----|--------|
| ReLU | max(0, z) | Veloce, no vanishing | Dead neurons |
| Sigmoid | 1/(1+e^-z) | Output [0,1] | Vanishing gradient |
| Tanh | (e^z-e^-z)/(e^z+e^-z) | Zero-centered | Vanishing gradient |
| Leaky ReLU | max(0.01z, z) | No dead neurons | Ancora lineare per z<0 |

---

## Template MLP minimale

```python
import numpy as np

class MLP:
    def __init__(self, input_dim, hidden_dim, output_dim, lr=0.1):
        # Xavier initialization
        self.W1 = np.random.randn(hidden_dim, input_dim) * np.sqrt(2/input_dim)
        self.b1 = np.zeros((hidden_dim, 1))
        self.W2 = np.random.randn(output_dim, hidden_dim) * np.sqrt(2/hidden_dim)
        self.b2 = np.zeros((output_dim, 1))
        self.lr = lr
    
    def relu(self, z): return np.maximum(0, z)
    def relu_grad(self, z): return (z > 0).astype(float)
    def sigmoid(self, z): return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def forward(self, X):
        self.z1 = self.W1 @ X + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = self.W2 @ self.a1 + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2
    
    def backward(self, X, y):
        m = X.shape[1]
        dz2 = self.a2 - y
        dW2 = (1/m) * dz2 @ self.a1.T
        db2 = (1/m) * np.sum(dz2, axis=1, keepdims=True)
        dz1 = (self.W2.T @ dz2) * self.relu_grad(self.z1)
        dW1 = (1/m) * dz1 @ X.T
        db1 = (1/m) * np.sum(dz1, axis=1, keepdims=True)
        # Update
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1
    
    def fit(self, X, y, epochs=1000):
        for _ in range(epochs):
            self.forward(X)
            self.backward(X, y)
```

---

## Errori comuni e soluzioni

| Errore | Conseguenza | Soluzione |
|--------|-------------|-----------|
| Learning rate troppo alto | Loss esplode/oscilla | Iniziare con 0.01, ridurre se instabile |
| Nessuna normalizzazione input | Convergenza lenta | Standardizzare X prima del training |
| Inizializzazione pesi grande | Saturazione attivazioni | Xavier o He initialization |
| Nessuna baseline | Non sai se MLP serve | Confronta sempre con LogisticRegression |
| Troppi neuroni su dati piccoli | Overfitting | Meno neuroni o regularizzazione |

---

## Metodi e concetti chiave

| Elemento | Ruolo |
|----------|-------|
| Forward pass | Calcola predizioni |
| Backward pass | Calcola gradienti |
| Chain rule | Propaga errore attraverso layer |
| Loss function | Misura distanza target-predizione |
| Gradient descent | Aggiorna pesi verso minimo |


# 7) Checklist di fine lezione
- [ ] Ho definito architettura e attivazioni.
- [ ] Ho verificato forme delle matrici e stabilita' del training.
- [ ] Ho confrontato MLP con baseline lineare.
- [ ] Ho monitorato loss/accuracy durante il training.
- [ ] Ho valutato il rischio di overfitting con piu' neuroni/epoche.

Glossario
- MLP: multilayer perceptron.
- ReLU/tanh: funzioni di attivazione non lineari.
- Backpropagation: algoritmo per calcolare i gradienti.
- Loss logistica: misura l'errore per classificazione binaria.


# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-01-XX | Struttura iniziale 8 sezioni |
| 2.0 | 2024-12-XX | Espansione completa Deep Learning |
| 2.1 | - | Architettura MLP con diagramma ASCII |
| 2.2 | - | Training loop visualizzato (forward/backward) |
| 2.3 | - | Spiegazione "perche' deep funziona" |
| 2.4 | - | Confronto attivazioni (ReLU, sigmoid, tanh) |
| 2.5 | - | Classe MLP template completa con backprop |
| 2.6 | - | Errori comuni: LR, init, baseline |

---

## Note di versione

**v2.0 - Espansione didattica completa**
- Visualizzazione architettura MLP con flusso dati
- Backpropagation spiegata come chain rule
- Emphasis su confronto con baseline lineare
- Template MLP riutilizzabile con Xavier init
- Preparazione concettuale per framework (PyTorch/TF)

**Dipendenze didattiche**
- Richiede: Lezione 06 (regressione logistica), calcolo derivate
- Prepara: Lezione 38 (modelli generativi), reti piu' complesse
